In [5]:
! pip install selenium beautifulsoup4

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import json
import csv
import re

# Your LinkedIn credentials
EMAIL = "nithishraju13@gmail.com"  # Replace with your LinkedIn email
PASSWORD = "Nithishr@43"        # Replace with your LinkedIn password

# Path to ChromeDriver
CHROMEDRIVER_PATH ='C:/Users/Sanjana SS/Downloads/chromedriver-win64 (2)/chromedriver-win64/chromedriver.exe'
# Initialize Selenium WebDriver
options = Options()
options.add_argument("--start-maximized")
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=options)

# Function to log in to LinkedIn
def linkedin_login(driver, email, password):
    driver.get("https://www.linkedin.com/login")
    time.sleep(5)

    email_field = driver.find_element(By.ID, "username")
    password_field = driver.find_element(By.ID, "password")

    email_field.send_keys(email)
    password_field.send_keys(password)

    login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    login_button.click()

    time.sleep(10)  # Increased wait time for the page to load fully

# Function to scroll LinkedIn feed
def scroll_feed(driver, scroll_count):
    for _ in range(scroll_count):
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)  # Scroll to the bottom
        time.sleep(7)  # Increased wait time for new posts to load

# Function to clean and convert likes count to integer
def convert_likes(likes_text):
    try:
        likes_text = likes_text.replace(",", "").strip()  # Remove commas from numbers like "1,192"
        return int(likes_text)
    except ValueError:
        return 0  # Default to 0 if conversion fails

# Function to scrape post content and engagement
def scrape_posts(driver):
    posts = []

    # Locate posts on the feed
    post_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'feed-shared-update-v2')]")
    for post in post_elements:
        try:
            # Extract post content
            content_element = post.find_element(By.XPATH, ".//span[contains(@class, 'break-words')]")
            content = content_element.text.strip()

            # Extract likes
            likes_element = post.find_element(By.XPATH, ".//span[contains(@class, 'social-details-social-counts__reactions-count')]")
            likes_text = likes_element.text.strip()
            engagement = convert_likes(likes_text)  # Convert likes to integer

            # Add data to the list
            posts.append({
                "text": content,
                "engagement": engagement
            })

        except Exception as e:
            print(f"Error scraping a post: {e}")
            continue

    return posts

# Save data as JSON
def save_as_json(posts):
    with open("linkedin_posts.json", "w", encoding="utf-8") as json_file:
        json.dump(posts, json_file, indent=4, ensure_ascii=False)
    print("Data saved to linkedin_posts.json")

# Save data as CSV
def save_as_csv(posts):
    with open("linkedin_posts.csv", "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=["text", "engagement"])
        writer.writeheader()
        writer.writerows(posts)
    print("Data saved to linkedin_posts.csv")

# Main Script
try:
    linkedin_login(driver, EMAIL, PASSWORD)  # Log in to LinkedIn
    scroll_feed(driver, scroll_count=10)     # Scroll feed to load more posts
    posts = scrape_posts(driver)             # Scrape posts

    # Save scraped data
    save_as_json(posts)  # Save as JSON
    save_as_csv(posts)   # Save as CSV

    print(f"Scraped {len(posts)} posts successfully!")

finally:
    driver.quit()  # Close the browser



Error scraping a post: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//span[contains(@class, 'social-details-social-counts__reactions-count')]"}
  (Session info: chrome=133.0.6943.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60072FBE5+28741]
	(No symbol) [0x00007FF600697890]
	(No symbol) [0x00007FF600534FDA]
	(No symbol) [0x00007FF600589322]
	(No symbol) [0x00007FF60058954C]
	(No symbol) [0x00007FF60057BF7C]
	(No symbol) [0x00007FF6005AFF4F]
	(No symbol) [0x00007FF60057BE46]
	(No symbol) [0x00007FF6005B0120]
	(No symbol) [0x00007FF6005D00B4]
	(No symbol) [0x00007FF6005AFCB3]
	(No symbol) [0x00007FF600579FB3]
	(No symbol) [0x00007FF60057B331]
	GetHandleVerifier [0x00007FF600A6A73D+3414941]
	GetHandleVerifier [0x00007FF600A7E64A+3496618]
	GetHandleVerifier [0x00007FF600A7413D+3454365]
	GetHandleVerifier [0

In [7]:
import json
import csv

# Function to remove duplicate posts based on text content
def remove_duplicates(posts):
    unique_posts = []
    seen_texts = set()

    for post in posts:
        text = post["text"].strip()  # Normalize text
        if text not in seen_texts:
            seen_texts.add(text)  # Add to seen set
            unique_posts.append(post)  # Add to new list

    return unique_posts

# Load the existing scraped posts from JSON
def load_posts_from_json(filename="linkedin_posts.json"):
    try:
        with open(filename, "r", encoding="utf-8") as json_file:
            return json.load(json_file)
    except FileNotFoundError:
        print(f"Error: {filename} not found.")
        return []

# Save the cleaned list as a JSON file
def save_as_json(posts, filename="raw_posts.json"):
    with open(filename, "w", encoding="utf-8") as json_file:
        json.dump(posts, json_file, indent=4, ensure_ascii=False)
    print(f"Data saved to {filename}")

# Save the cleaned list as a CSV file
def save_as_csv(posts, filename="raw_posts.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=["text", "engagement"])
        writer.writeheader()
        writer.writerows(posts)
    print(f"Data saved to {filename}")

# Main Execution
if __name__ == "__main__":
    # Load scraped posts
    scraped_posts = load_posts_from_json()

    if scraped_posts:
        # Remove duplicates
        filtered_posts = remove_duplicates(scraped_posts)

        # Save cleaned data
        save_as_json(filtered_posts, "raw_posts.json")
        save_as_csv(filtered_posts, "raw_posts.csv")

        print(f"Total unique posts saved: {len(filtered_posts)}")
    else:
        print("No data found to process.")


Data saved to raw_posts.json
Data saved to raw_posts.csv
Total unique posts saved: 5
